In [1]:
import os
import kagglehub

In [2]:
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
with_mask_path = '/kaggle/input/face-mask-dataset/data/with_mask'
without_mask_path = '/kaggle/input/face-mask-dataset/data/without_mask'

In [5]:
with_mask_images = [os.path.join(with_mask_path, i) for i in os.listdir(with_mask_path)]
without_mask_images = [os.path.join(without_mask_path, i) for i in os.listdir(without_mask_path)]

In [6]:
with_mask_labels = [1] * len(with_mask_images)
without_mask_labels = [0] * len(without_mask_images)

In [7]:
images = with_mask_images + without_mask_images
labels = with_mask_labels + without_mask_labels
df = pd.DataFrame({'files': images, 'class': labels})
df['class'] = df['class'].astype(str)

In [8]:
df_train, df_temp = train_test_split(df, test_size=0.3, stratify=df['class'], random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['class'], random_state=42)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.25,
    brightness_range=[0.7,1.3],
    horizontal_flip=True,
    fill_mode='nearest'
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

Found 5287 validated image filenames belonging to 2 classes.


In [11]:
val_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_val,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 1133 validated image filenames belonging to 2 classes.


In [12]:
test_gen = test_val_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='files',
    y_col='class',
    target_size=(128, 128),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 1133 validated image filenames belonging to 2 classes.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , Dropout, BatchNormalization , Activation
from tensorflow.keras.regularizers import l2

In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(0.001) , padding='same' , input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(0.001) , padding='same' , input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

In [17]:
model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(0.005) , padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

In [18]:
model.add(Conv2D(128, (3, 3), kernel_regularizer=l2(0.005) , padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))

In [19]:
model.add(Flatten())
model.add(Dense(256, kernel_regularizer=l2(0.005)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
from keras.callbacks  import EarlyStopping

In [22]:
earlystop = EarlyStopping(patience=2 , restore_best_weights=True)

In [23]:
model.fit(train_gen, validation_data=val_gen, epochs=30 ,  callbacks=[earlystop])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
107/166 ━━━━━━━━━━━━━━━━━━━━ 20s 345ms/step - accuracy: 0.7053 - loss: 3.5203

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


166/166 ━━━━━━━━━━━━━━━━━━━━ 86s 439ms/step - accuracy: 0.7286 - loss: 3.2976 - val_accuracy: 0.5305 - val_loss: 2.1130
Epoch 2/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 217ms/step - accuracy: 0.8091 - loss: 1.6122 - val_accuracy: 0.6178 - val_loss: 1.4252
Epoch 3/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 214ms/step - accuracy: 0.8232 - loss: 1.0526 - val_accuracy: 0.5269 - val_loss: 1.5367
Epoch 4/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 36s 214ms/step - accuracy: 0.8461 - loss: 0.8439 - val_accuracy: 0.6884 - val_loss: 1.1530
Epoch 5/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 37s 220ms/step - accuracy: 0.8506 - loss: 0.7568 - val_accuracy: 0.7617 - val_loss: 0.8660
Epoch 6/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 35s 210ms/step - accuracy: 0.8574 - loss: 0.6916 - val_accuracy: 0.7873 - val_loss: 0.7599
Epoch 7/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 42s 215ms/step - accuracy: 0.8772 - loss: 0.6429 - val_accuracy: 0.7988 - val_loss: 0.7144
Epoch 8/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 41s 215ms/step - accuracy: 0.8787 - loss: 0.6011 - val

In [24]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc*100:.2f}%")

36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 188ms/step - accuracy: 0.9479 - loss: 0.4975
Test Accuracy: 94.00%


In [27]:
model.save('face_mask_detection.h5')